# DEMO sprint 2 - Deel 3 (belastingen classificeren)

Gebruikmakend van:
- csv bestanden voor uitwisselen van data

### Benodigde generieke Python packages

In [ ]:
from pathlib import Path

### Definitie configuratie: Database bestanden 

GlobalVariables:
    rootdir: "data_demo" 
    moments: [-24,0,24,48]  

    LoadsMaxima:
        tide: False   

DataAdapter: 
    default_options:
        csv:
            sep: ","
    Waterstanden_WaterInfo: 
        type: csv
        path: "waterstanden_waterinfo.csv"
    Waterstanden_WaterInfo_Maxima:   
        type: csv
        path: "waterstanden_waterinfo_maxima.csv"    

### Lees configuratie in en maak dictionary base.Config aan

In [ ]:
from toolbox_continu_inzicht import Config

yaml_config_file = "demo_sprint2-III.yaml"
path = Path.joinpath(Path.cwd(), "data_demo", yaml_config_file)
c = Config(config_path=path)
c.lees_config()

### Geef config mee aan base.Adapter

In [ ]:
from toolbox_continu_inzicht import DataAdapter

data = DataAdapter(config=c)
print(f"Folder met data: {data.config.config_path}")
print(f"Globale variabelen: {data.config.global_variables}")
for data_adapter in data.config.data_adapters:
    print(f"Data apdater: {data_adapter}")

### Invoerbestand (CSV) met data van waterstanden per belastinglocatie
Verplichte velden (schema):
- measurement_location_id: int64
- measurement_location_code: object (string)
- measurement_location_description: object (string)
- parameter_id: int64
- parameter_code: object (string)
- parameter_description: object (string)
- unit: object (string)
- date_time: object (datetime)
- value: float
- value_type: object (string)

### Laad en run module (functie) LoadsToMoments voor inlezen waterstand reeksen en bepalen waterstanden per moment

In [ ]:
from toolbox_continu_inzicht.loads import LoadsToMoments

moments = LoadsToMoments(data_adapter=data)
ds = moments.run(input="Waterstanden_WaterInfo", output="Waterstanden_WaterInfo_Maxima")

In [ ]:
waterlevels = moments.df_in
waterlevels

In [ ]:
waterlevels_moments = moments.df_out
waterlevels_moments

### Laad module (functie) LoadsClassify voor bepalen classificatie van waterstanden per moment

In [ ]:
from toolbox_continu_inzicht.loads import LoadsClassify

classify = LoadsClassify(data_adapter=data)

### Run module (functie) LoadsClassify voor bepalen classificatie van waterstanden per moment
LET OP: volgorde van de input!!! Eerst de klassegrenzen en daarna de waterstanden

Klassegrenzen:\
measurement_location_id,lower_boundary,upper_boundary,color,label\
1,0.0,695.0,"#daeef3","normaal"\
1,695.0,1155.0,"#b6dde8","licht verhoogd"\
1,1155.0,1250.0,"#4bacc6","verhoogd"\
1,1250.0,1430.0,"#31849b","hoog water"\
1,1430.0,10000.0,"#002060","extreem hoog water"

In [ ]:
ds = classify.run(
    input=["WaterInfo_Klassengrenzen", "Waterstanden_WaterInfo_Maxima"],
    output="Waterstanden_WaterInfo_Klassen",
)

In [ ]:
waterlevels_classes = classify.df_out
waterlevels_classes

<!-- values_nan = df_out[df_out["value"] == -999].index
for val in values_nan:
    df_out.loc[val, "value"] = np.nan
df_out.sort_index()
df_plot = df_out.set_index(df_out["datetime"])
df_plot = df_plot[df_plot["measurement_location_id"] == 8420]
measurement_index = df_plot["value_type"] == "verwachting"
df_plot_measurements = df_plot[measurement_index][["value"]]
df_plot_forecast = df_plot[~measurement_index][["value"]]
fig, ax = plt.subplots()
df_plot_measurements.plot(color="C0", ax=ax)
df_plot_forecast.plot(color="C1", ax=ax)
ax.legend(["Prediction", "Measurement"]) -->